In [ ]:
import copy
from pathlib import Path
import warnings

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet, DeepAR, DecoderMLP, RecurrentNetwork, NBeats
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss, RMSE, MAPE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_forecasting.data import NaNLabelEncoder

from sklearn.preprocessing import MinMaxScaler
import random
import os

def set_seed(seed):
    # seed init.
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    # torch seed init.
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False # train speed is slower after enabling this opts.

    # https://pytorch.org/docs/stable/generated/torch.use_deterministic_algorithms.html
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

    # avoiding nondeterministic algorithms (see https://pytorch.org/docs/stable/notes/randomness.html)
    torch.use_deterministic_algorithms(True, warn_only=True)

set_seed(42)

In [ ]:
#Before making predictions, it is necessary to remove autocorrelation, such as converting it into volatility or return. In Github, we only provide the raw data.
data = pd.read_csv('return_data_matrix')
data.head(50)

data = data.astype(dict(series=str))

#Train, Validation, Test Split
n = len(data)
training_df = data[0:int(n * 0.8)]
validation_df = data[int(n * 0.8):int(n * 0.9)]
test_df = data[int(n * 0.9):]

max_prediction_length = 1
max_encoder_length = 5
#training_cutoff = training_df["time_idx"].max() - max_prediction_length*2

training = TimeSeriesDataSet(
    training_df,
    #training_df[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target=['high','low'],
    group_ids=["series"],
    min_encoder_length=max_encoder_length,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,

    max_prediction_length=max_prediction_length,
    categorical_encoders={"series": NaNLabelEncoder().fit(training_df.series)},
    static_categoricals=["series"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "high",
        "low",
        "close",
        "open",
        "volumek",
        "gas",
        'oil',
        'coal',
        'power',
        'stock',
        'pos',
        'neg',
        'trends'
    ],
    add_relative_time_idx=True,
)


validation = TimeSeriesDataSet.from_dataset(training, validation_df, predict=False, stop_randomization=True)
test = TimeSeriesDataSet.from_dataset(training, test_df, predict=False, stop_randomization=True)

# create dataloaders for model
batch_size = 64  # set this between 32 to 128

train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)
test_dataloader = test.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

# and load the first batch
x, y = next(iter(time_dataloader))
print("x =", x)
print("\ny =", y)
print("\nsizes of x =")
for key, value in x.items():
    print(f"\t{key} = {value.size()}")                              

In [ ]:
import pickle
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# create study
study = optimize_hyperparameters(
    train_dataloader,
    train_dataloader,
    model_path="optuna_test",
    n_trials=30,
    max_epochs=300,
    gradient_clip_val_range=(0.1, 0.1),
    hidden_size_range=(8, 256),
    hidden_continuous_size_range=(4, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.001, 0.1),
    dropout_range=(0.1, 0.3),
    trainer_kwargs=dict(limit_train_batches=64, accelerator="cpu"),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False,  # use Optuna to find ideal learning rate or use in-built learning rate finder
)
# save study results - also we can resume tuning at a later point in time
with open("test_study.pkl", "wb") as fout:
    pickle.dump(study, fout)
    
# show best hyperparameters
print(study.best_trial.params)

In [ ]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="train_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

pl.seed_everything(42)
trainer = pl.Trainer(
    max_epochs=1000,
    accelerator="cpu",
    enable_model_summary=True,
    gradient_clip_val=0.1,
    limit_train_batches=64,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=study.best_trial.params['learning_rate'],
    hidden_size=study.best_trial.params['hidden_size'],
    attention_head_size=study.best_trial.params['attention_head_size'],
    dropout=study.best_trial.params['dropout'],
    hidden_continuous_size=study.best_trial.params['hidden_continuous_size'],
    loss=QuantileLoss(),
    #log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    optimizer="Ranger",
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


#{'gradient_clip_val': 0.1, 'hidden_size': 127, 'dropout': 0.1996489099681776, 'hidden_continuous_size': 61, 'attention_head_size': 3, 'learning_rate': 0.009964460308051648}
#{'gradient_clip_val': 0.1, 'hidden_size': 79, 'dropout': 0.26240598317466385, 'hidden_continuous_size': 25, 'attention_head_size': 2, 'learning_rate': 0.0077954926055157}
#{'gradient_clip_val': 0.1, 'hidden_size': 30, 'dropout': 0.12401596219667392, 'hidden_continuous_size': 18, 'attention_head_size': 4, 'learning_rate': 0.009616917298488703}

In [ ]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="train_loss", min_delta=1e-5, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

pl.seed_everything(42, workers=True)
#pl.Trainer(deterministic=True)
trainer = pl.Trainer(
    max_epochs=300,
    accelerator="cpu",
    enable_model_summary=True,
    gradient_clip_val=0.1,
    limit_train_batches=64,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,

)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.07,
    hidden_size=115,
    attention_head_size=2,
    dropout=0.1,
    hidden_continuous_size=44,
    loss=QuantileLoss(),
    #log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    optimizer="Ranger",
    reduce_on_plateau_patience=4,

)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


In [ ]:
# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
actual_low = np.array(predictions.x['decoder_target'][1].cpu())
actual_high = np.array(predictions.x['decoder_target'][0].cpu())
pred_low = np.array(predictions.output[1].cpu())
pred_high= np.array(predictions.output[0].cpu())

In [ ]:
# Function to calculate the directional accuracy for interval predictions
def interval_Dstat(actual_low, actual_high, pred_low, pred_high):
    # Calculate the number of intervals
    n = len(actual_low)
    # Calculate the directional accuracy
    correct_predictions = (np.array(actual_high * pred_high > 0) & np.array(actual_low * pred_low > 0)).astype(int)
    interval_Dstat = np.sum(correct_predictions) / n  # n-1 because of np.diff reducing the length by 1
    return interval_Dstat

# Function to calculate interval evaluation metrics
def IMAPE(actual_low, actual_high, pred_low, pred_high):
    n = len(actual_low)
    imape = (1 / (2 * n)) * np.sum(np.abs((actual_low - pred_low) / actual_low) + np.abs((actual_high - pred_high) / actual_high))
    return imape

def IRMSE(actual_low, actual_high, pred_low, pred_high):
    n = len(actual_low)
    irmse = 0.5 * (np.sqrt((1 / n) * np.sum((actual_low - pred_low) ** 2)) + np.sqrt((1 / n) * np.sum((actual_high - pred_high) ** 2)))
    return irmse

def IARV(actual_low, actual_high, pred_low, pred_high):
    n = len(actual_low)
    x_mean_low = np.mean(actual_low)
    x_mean_high = np.mean(actual_high)
    numerator = np.sum((actual_low[1:] - pred_low[:-1]) ** 2) + np.sum((actual_high[1:] - pred_high[:-1]) ** 2)
    denominator = np.sum((actual_low[1:] - x_mean_low) ** 2) + np.sum((actual_high[1:] - x_mean_high) ** 2)
    iarv = numerator / denominator
    return iarv

def UI(actual_low, actual_high, pred_low, pred_high):
    n = len(actual_low)
    x_mean_low = np.mean(actual_low)
    x_mean_high = np.mean(actual_high)
    numerator = np.sum((actual_low[1:] - pred_low[:-1]) ** 2) + np.sum((actual_high[1:] - pred_high[:-1]) ** 2)
    denominator = np.sum((actual_low[1:] - x_mean_low) ** 2) + np.sum((actual_high[1:] - x_mean_high) ** 2)
    ui = np.sqrt(numerator / denominator)
    return ui



# Function to calculate interval metrics
def interval_metrics(actual_low, actual_high, pred_low, pred_high):
    metrics = {
        'IMAPE': IMAPE(actual_low, actual_high, pred_low, pred_high),
        'IRMSE': IRMSE(actual_low, actual_high, pred_low, pred_high),
        'IARV': IARV(actual_low, actual_high, pred_low, pred_high),
        'UI': UI(actual_low, actual_high, pred_low, pred_high),
        # 'Directional Accuracy High': Dstat(actual_high, pred_high),
        # 'Directional Accuracy Low': Dstat(actual_low, pred_low),
        'interval_Dstat':interval_Dstat(actual_low, actual_high, pred_low, pred_high)
    }
    return metrics

# Calculate and print the interval metrics
metrics = interval_metrics(actual_low, actual_high, pred_low, pred_high)
for metric, value in metrics.items():
    print(f'{metric}: {value:.4f}')

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
attentions = best_tft.predict(test_dataloader, mode="raw", return_x=True)

interpretation = best_tft.interpret_output(attentions.output, reduction="sum")
best_tft.plot_interpretation(interpretation)

print(interpretation['encoder_variables'])